Libraries

In [36]:
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import librosa
import scipy.io.wavfile as wave
from scipy.fftpack import fft, ifft, fftshift
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import mixture
import joblib
import pandas as pd
from sklearn.mixture import GaussianMixture
import json
from scipy.io import loadmat

Variables

In [37]:

n_mixtures = 8
max_iterations = 1000
#calc_deltas=True
#sr=8000
#hop_length=int(0.005*sr)
home_path = os.getcwd()
#path = os.path.join(home_path, 'data')
print(home_path)

/home/tarun/snap/SSP-project


Extraction 

In [38]:
def extract_features(file):
    feature = loadmat(file)
    
    dur_list = np.array([[element for element in upperElement] for upperElement in feature['dur']])
    f0_list = np.array([[element for element in upperElement] for upperElement in feature['f0']])
    amp_tilt_list = np.array([[element for element in upperElement] for upperElement in feature['amp_tilt']])
    dur_tilt_list = np.array([[element for element in upperElement] for upperElement in feature['dur_tilt']])
    differ_list = np.array([[element for element in upperElement] for upperElement in feature['differ']])
    voiced_dur_list = np.array([[element for element in upperElement] for upperElement in feature['voiced_dur']])
    log_energy_list = np.array([[element for element in upperElement] for upperElement in feature['log_energy']])
    
    dur_list = dur_list.ravel()
    f0_list = f0_list.ravel()
    amp_tilt_list = amp_tilt_list.ravel()
    dur_tilt_list = dur_tilt_list.ravel()
    differ_list = differ_list.ravel()
    voiced_dur_list = voiced_dur_list.ravel()
    log_energy_list = log_energy_list.ravel()
    
    feature_vec = np.vstack((dur_list,f0_list,amp_tilt_list,dur_tilt_list,differ_list,voiced_dur_list,log_energy_list))
    feature_vec = np.nan_to_num(feature_vec)
    feature_vec = feature_vec.T
    return feature_vec
    #print(feature_vec)

Training Function

In [39]:
def training(train_data_path,feat_train_path,trained_model_path):
    all_speakers=glob.glob(train_data_path+'*')
    #print(all_speakers)
    print(2)

    directory=feat_train_path
    if not os.path.exists(directory):
        os.makedirs(directory)
      
    directory=trained_model_path
    if not os.path.exists(directory):
        os.makedirs(directory)

    print(len(all_speakers))
    for itr1 in range(0,len(all_speakers)):
        
        mats=glob.glob(all_speakers[itr1]+'/*.mat')
        spk=(all_speakers[itr1]).split("/")[-1]
        #print((mats))

        if not os.path.exists(directory):
            os.makedirs(directory)
      
        final_feat=np.empty([0, 35])
        
        for itr2 in range(0,len(mats)):
  
            #final_feat = eng.prosody(mats[itr2])
            #mat file
            #mats[itr2]=mats[itr2].replace("\\","/")
            #print(mats[itr2])
            final_feat = extract_features(mats[itr2])
            #print(final_feat.shape)

        #print(spk)    
        np.savetxt(feat_train_path + spk + "_all_features.txt" , final_feat , delimiter=",")
        try:
            gmm = mixture.GaussianMixture(n_components=n_mixtures, covariance_type='diag' , max_iter = max_iterations ).fit(final_feat)
        except:
            print("ERROR : Error while training model for file "+spk+mats[itr2])
          
        try:
            joblib.dump(gmm,trained_model_path+spk+'.pkl')
        except:
            print("ERROR : Error while saving model for "+spk)
          

    print("Training Completed")


Testing Function

In [40]:
def testing(test_data_path,feat_test,trained_model_path):
    # train feature extraction
    all_speakers=glob.glob(test_data_path+'*')

    import os
    directory=feat_test
    if not os.path.exists(directory):
        os.makedirs(directory)

    speakers = { all_speakers[k]:k for k in range(len(all_speakers)) }
    print(speakers) 
   
    num_test_cases={}
    tct={}
    for e in speakers:
        num_test_cases[e.replace(test_data_path,'')]=len(os.listdir(e))-1
        tct[e.replace(test_data_path,'')]=0

    # print(num_test_cases)

    spk_names = { all_speakers[k].replace(test_data_path,''):k for k in range(len(all_speakers)) }
    total_speakers=len(num_test_cases)
    confusion_matrix = np.zeros((total_speakers,total_speakers))


    for itr1 in range(0,len(all_speakers)):
        mats=glob.glob(all_speakers[itr1]+'/*.mat')
        spk=(all_speakers[itr1]).split("/")[-1]
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        final_feat=np.empty([0, 35])
        output = {"Test_i":[],"Accent":[]}

        for itr2 in range(0,len(mats)):
            print(mats[itr2])

            feat = extract_features(mats[itr2])
            #print(feat)
            max_score=-np.inf
            max_spk_name=""

            for modelfile in sorted(glob.glob(trained_model_path+'*.pkl')):
                gmm = joblib.load(modelfile) 
                score=gmm.score(feat)
                #print score
                if score>max_score:
                    max_score,max_spk_name=score,modelfile.replace(trained_model_path,'').replace('.pkl','')
            try:
                print(spk+" -> "+max_spk_name+(" Y" if spk==max_spk_name  else " N"))
                output ["Test_i"].append(mats[itr2])
                output ["Accent"].append(max_spk_name)
                confusion_matrix[ spk_names[spk] ][spk_names[max_spk_name]]+=1
                tct[spk]+=1
            except:
                print("ERROR : Error while testing file "+spk+mats[itr2])

        #print(spk)
        json_object = json.dumps(output, indent = 4)

        with open("Output.json", "w") as outfile:
              outfile.write(json_object)
        np.savetxt(feat_test+spk+"_all_features.txt", feat, delimiter=",")
        
    return tct,confusion_matrix,total_speakers


File Paths

In [41]:
cwd = os.getcwd()
feat= './feat/'
feat_train=  './feat/train/'
feat_test= './feat/test/'
trained_model= './trained_model/'
train_data= './traindata/'
test_data= './testdata/'
print(feat)
# for removing existing feature folders, models created
if os.path.exists('./feat/'):
  !rm -rf  './feat/'
if os.path.exists('./feat/test/'):
  !rm -rf  './feat/test/'

./feat/


Training

In [42]:
training(train_data,feat_train,trained_model)

2
7
ERROR : Error while training model for file bengali./traindata/bengali/bengalif25.mat
Training Completed


Testing

In [43]:
tt,conf_mat,tot_spek=testing(test_data,feat_test,trained_model)

{'./testdata/manipur': 0, './testdata/telugu': 1, './testdata/bengali': 2, './testdata/odia': 3, './testdata/assam': 4, './testdata/gujrati': 5, './testdata/marathi': 6}
./testdata/manipur/manipurf6.mat
manipur -> manipur Y
./testdata/manipur/manipurf7.mat
manipur -> manipur Y
./testdata/manipur/manipurf2.mat
manipur -> manipur Y
./testdata/manipur/manipurf1.mat
manipur -> manipur Y
./testdata/manipur/manipurf8.mat
manipur -> manipur Y
./testdata/manipur/manipurf9.mat
manipur -> manipur Y
./testdata/manipur/manipurf10.mat
manipur -> manipur Y
./testdata/manipur/manipurf3.mat
manipur -> manipur Y
./testdata/manipur/manipurf11.mat
manipur -> manipur Y
./testdata/manipur/manipurf5.mat
manipur -> manipur Y
./testdata/telugu/telugu9.mat
telugu -> bengali N
./testdata/telugu/telugu5.mat
telugu -> manipur N
./testdata/telugu/telugu6.mat
telugu -> bengali N
./testdata/telugu/telugu8.mat
telugu -> bengali N
./testdata/telugu/telugu10.mat
telugu -> odia N
./testdata/telugu/telugu1.mat
telugu -> 

Accuracy

In [44]:
print(tt)
print("Confusion Matrix:\n",conf_mat)
print("Total Speakers:",tot_spek)
print("Accuracy: ",(sum([ conf_mat[i][j] if i==j  else 0 for i in range(tot_spek) for j in range(tot_spek) ] )*100)/float(sum([i for i in tt.values()])))

{'manipur': 10, 'telugu': 9, 'bengali': 10, 'odia': 7, 'assam': 9, 'gujrati': 10, 'marathi': 10}
Confusion Matrix:
 [[10.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  5.  3.  0.  0.  0.]
 [ 4.  0.  5.  1.  0.  0.  0.]
 [ 0.  0.  0.  7.  0.  0.  0.]
 [ 0.  0.  0.  9.  0.  0.  0.]
 [ 7.  0.  0.  3.  0.  0.  0.]
 [ 8.  0.  1.  1.  0.  0.  0.]]
Total Speakers: 7
Accuracy:  33.84615384615385
